Necessary Imports

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Flatten, Dense, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

Custom Layer for weights and bias

In [19]:
class CustomConv2D(Layer):
    def __init__(self, filters, kernel_size, activation='relu'):
        super(CustomConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.activation = tf.keras.activations.get(activation)

# kernal - filter, filter - number of kernal
    def build(self, input_shape):
        self.kernel = self.add_weight(
            
            shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
            initializer='glorot_uniform',
            trainable=True,
            name='kernel'

        )

        self.bias = self.add_weight(
            shape=(self.filters,),
            initializer='zeros',
            trainable=True,
            name='bias'
        )

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output


custom Dense Layer

In [20]:
class CustomDense(Layer):
    def __init__(self, units, activation='relu'):
        super(CustomDense, self).__init__()
        self.units = units
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.weight = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer='glorot_uniform',  
            trainable=True,
            name='weights'
        )

        self.bias = self.add_weight(
            shape=(self.units,),
            initializer='zeros',  
            trainable=True,
            name='bias'
        )

    def call(self, inputs):
        output = tf.matmul(inputs, self.weight)
        output = tf.nn.bias_add(output, self.bias)
        output = self.activation(output)
        return output


CustomModel

In [21]:
class CustomModel(Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = CustomConv2D(3, (3,3))
        self.pool1 = MaxPooling2D((2,2))
        self.conv2 = CustomConv2D(6, (6,6))
        self.pool2 = MaxPooling2D((2,2))  
        self.flatten = Flatten()
        self.dense1  = CustomDense(64)
        self.dense2 = CustomDense(32)
        self.dense3 = CustomDense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)  # Corrected call to pool2
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = self.dense3(x)
        return output


In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator for normalizing pixel values
normalize_train = ImageDataGenerator(rescale=1./255)
normalize_test = ImageDataGenerator(rescale=1./255)

# Load train data using ImageDataGenerator
train_data = normalize_train.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\train",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary'    
)

# Load test data using ImageDataGenerator
test_data = normalize_test.flow_from_directory(
    r"C:\Users\aethe\Desktop\work\EDA\dataset\test",
    target_size=(150, 150),
    batch_size=15,
    class_mode='binary'
)



Found 20 images belonging to 2 classes.
Found 6 images belonging to 2 classes.


In [35]:
model = CustomModel()

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=['accuracy'])

# Train the model
model.fit(train_data, epochs=5, validation_data=test_data)


Epoch 1/5

2/2 [==============================] - 1s 254ms/step - loss: 0.6973 - accuracy: 0.3500 - val_loss: 0.6801 - val_accuracy: 0.8333
Epoch 2/5
2/2 [==============================] - 0s 75ms/step - loss: 0.6905 - accuracy: 0.6000 - val_loss: 0.6762 - val_accuracy: 0.6667
Epoch 3/5
2/2 [==============================] - 0s 100ms/step - loss: 0.6938 - accuracy: 0.5000 - val_loss: 0.6715 - val_accuracy: 0.5000
Epoch 4/5
2/2 [==============================] - 0s 88ms/step - loss: 0.6743 - accuracy: 0.5000 - val_loss: 0.6626 - val_accuracy: 0.5000
Epoch 5/5
2/2 [==============================] - 0s 73ms/step - loss: 0.6694 - accuracy: 0.5000 - val_loss: 0.6602 - val_accuracy: 0.5000


In [36]:
model.evaluate(test_data)

1/1 [==============================] - 0s 55ms/step - loss: 0.6602 - accuracy: 0.5000


[0.660244345664978, 0.5]